In [1]:
import numpy as np, random, operator, pandas as pd, math

class Direction:
    def __init__(self, input_size, hidden_size, output_size):
        self.w1 = np.random.uniform(-1., 1., (input_size,hidden_size))
        self.w2 = np.random.uniform(-1., 1., (hidden_size,hidden_size))
        self.w3 = np.random.uniform(-1., 1., (hidden_size,output_size))
        
    def __repr__(self):
        return str(self.w1) + ',' + str(self.w2) + ',' + str(self.w3)
        
    def getWeights(self):
        return self.w1, self.w2, self.w3
    
    
class Results:
    def __init__(self):
        self.winners = 0
        self.generations = 0
        
    def incrementWinners(self, winner):
        self.winners += winner
        
    def getWinners(self):
        return str(self.winners)
    
    def resetWinners(self):
        self.winners = 0
    
    def incrementGenerations(self):
        self.generations += 1
        
    def getGenerations(self):
        return self.generations
    
class Fitness:
    def __init__(self, population):
        self.population = population
        self.bias = 1
        
        self.listSquares = []
        # 0 weights
        # 1 rect
        # 2 alive
        # 3 color
        # 4 distance
        
        self.fitnessResults = {}
        self.hasSquaresAlive = True
        self.bestDistance = 1000000
        
        
    def sigmoid(self, x, derivative=False):
        return x*(1.0-x) if derivative else 1.0/(1.0+np.exp(-x))
    
    # calcular a ação a ser executada seguindo a maior probabilidade
    def feedforward(self, individual, input=None):
        input_array = []

        if input is None:
            input_array = np.array(self.input, ndmin=2)
        else:
            input_array = np.array(input, ndmin=2)
        
        #xxx = np.dot(input_array, self.w1)
        
        #print('xxx', xxx)
        
        w1 = individual[0]
        w2 = individual[1]
        w3 = individual[2]
            
        layer1 = self.sigmoid(np.dot(input_array, w1) + self.bias)
        layer2 = self.sigmoid(np.dot(layer1, w2) + self.bias)
        output = self.sigmoid(np.dot(layer2, w3) + self.bias)
        
        return output;
        

        
    def calDistance(self):
        
        results.resetWinners()
        
        kill_line = pygame.Rect(0, 0, 3, 300)
        
        #print('position_square')
        #print(square.centerx, square.centery)
        
        collided = False
        
        #squares = self.population
        
        for i in range(0, len(self.population)):
            square = []
            
            square.append(self.population[i]) #weights
            square.append(pygame.Rect(50, 250, SQUARE_WIDTH, SQUARE_HEIGHT)) #rect
            square.append(True) #alive
            square.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))) #color
            square.append(0) #distance
            
            self.listSquares.append(square)

        
        while self.hasSquaresAlive == True:
            
            
            screen.fill(BLACK)
            
            dt = clock.tick(60)
            
            if kill_line.centerx == 600:
                kill_line.centerx = 0
                self.hasSquaresAlive = False
                continue

            kill_line.move_ip(dt * kill_line_velocity, 0)
            pygame.draw.rect(screen, RED, kill_line)
            
            for wall in walls:
                pygame.draw.rect(screen, WHITE, wall)


            for border in borders:
                pygame.draw.rect(screen, RED, border)

            pygame.draw.rect(screen, GREEN, exit)
            
            
            
            for index in range(0, len(self.listSquares)):
                
                if self.listSquares[index][2] == False:
                    continue
                
                distance_top = distance_bottom = distance_right = distance_left = 600
                
                closerRight = closerBottom = closerLeft = closerTop = 0
                
                for wall in walls:
                    
                    square = self.listSquares[index][1]
                    
                    # o quadrado é menor que as paredes, então ele sempre estará
                    # contido em uma parede, os testes abaixo verificam esse caso
                    
                    if square.left > wall.left and square.right < wall.right:
                        if square.top > wall.bottom:
                            if abs(square.top - wall.bottom) <= distance_top:
                                distance_top = abs(square.top - wall.bottom)
                                closerTop = wall

                        if square.bottom < wall.top:
                            if abs(square.bottom - wall.top) <= distance_bottom:
                                distance_bottom = abs(square.bottom - wall.top)
                                closerBottom = wall

                                
                    if square.top > wall.top and square.bottom < wall.bottom: 
                        if square.right < wall.left:
                            if abs(square.right - wall.left) <= distance_right:
                                distance_right = abs(square.right - wall.left)
                                closerRight = wall

                        if square.left > wall.right:
                            if abs(square.left - wall.right) <= distance_left:
                                distance_left = abs(square.left - wall.right)
                                closerLeft = wall
                        
                
                x1, y1 = square.center
                
                
                if closerRight != 0:
                    r_x2, r_y2 = closerRight.center
                    pygame.draw.line(screen, RED, (x1, y1), (r_x2, r_y2), 1)
                
                if closerBottom != 0:
                    b_x2, b_y2 = closerBottom.center
                    pygame.draw.line(screen, RED, (x1, y1), (b_x2, b_y2), 1)

                if closerLeft != 0: 
                    l_x2, l_y2 = closerLeft.center
                    pygame.draw.line(screen, RED, (x1, y1), (l_x2, l_y2), 1)

                if closerTop != 0:    
                    t_x2, t_y2 = closerTop.center
                    pygame.draw.line(screen, RED, (x1, y1), (t_x2, t_y2), 1)

                
                input_array = distance_right, distance_bottom, distance_left, distance_top

                output = self.feedforward(self.listSquares[index][0], input_array)
                
                #print('input_array')
                #print(input_array)
                
                #print('output')
                #print(output)

                action = np.argmax(output)

                #print('action')
                #print(action)

                #0 direita
                #1 baixo
                #2 esquerda
                #3 cima

                pos_x = pos_y = 0

                if (action == 0):
                    pos_x = velocity * dt

                if (action == 1):
                    pos_y = (velocity * dt)

                if (action == 2):
                    pos_x = -(velocity * dt)

                if (action == 3):
                    pos_y = -(velocity * dt)

                self.listSquares[index][1].move_ip(pos_x, pos_y)    
                pygame.draw.rect(screen, self.listSquares[index][3], self.listSquares[index][1])

                
                if self.listSquares[index][1].colliderect(kill_line):
                    self.listSquares[index][2] = False


                for wall in walls:
                    if self.listSquares[index][1].colliderect(wall):
                        #print('bateu')
                        self.listSquares[index][2] = False


                for border in borders:
                    if self.listSquares[index][1].colliderect(border):
                        #print('bateu')
                        self.listSquares[index][2] = False


                #init_x, init_y = WIDTH // 2, HEIGHT // 2
                init_x, init_y = exit.centerx, exit.centery
                current_x, current_y = self.listSquares[index][1].centerx, self.listSquares[index][1].centery

                xDis = abs(current_x - init_x)
                #calcula a distancia dos eixos y
                
                yDis = abs(current_y - init_y)
                #eleva as diferenças ao quadrado, soma e calcula a raiz
                
                distance = np.sqrt((xDis ** 2) + (yDis ** 2))
                
                if distance < self.bestDistance:
                    self.bestDistance = distance
                
                self.listSquares[index][4] = distance

                if self.listSquares[index][1].colliderect(exit):
                    results.incrementWinners(1)

                
            alives = 0

            for alive in self.listSquares:
                if alive[2] == True:
                    alives += 1
                    
            if (alives == 0):
                self.hasSquaresAlive = False
                
                
            generations_text = 'geracao: ' + str(results.getGenerations())
            generations_font = font.render(generations_text, True, WHITE)
            generations_rect = generations_font.get_rect()
            generations_rect.center = (WIDTH - 200, HEIGHT - 80)

            screen.blit(generations_font, generations_rect)
            

            alives_text = 'vivos: ' + str(alives)
            alives_font = font.render(alives_text, True, WHITE)
            alives_rect = alives_font.get_rect()
            alives_rect.center = (WIDTH - 200, HEIGHT - 60)

            screen.blit(alives_font, alives_rect)
            

            winners_text = 'vencedores: ' + str(results.getWinners())
            winners_font = font.render(winners_text, True, WHITE)
            winners_rect = winners_font.get_rect()
            winners_rect.center = (WIDTH - 200, HEIGHT - 40)

            screen.blit(winners_font, winners_rect)
            

            pygame.display.flip()


        return True
            
    def getPathFitness(self):
        
        self.calDistance()
        
        print('bestDistance', self.bestDistance)
        
        
        for index in range(0, len(self.listSquares)):
            distance = self.listSquares[index][4]
            
            fitness = 1 / distance
                
            self.fitnessResults[index] = fitness
            
        #print('fitnessResults')
        #print(self.fitnessResults)
        
        return self.fitnessResults

import pygame
import sys

BLACK = pygame.Color(0, 0, 0)
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0)
GREEN = pygame.Color(0, 255, 0)
WIDTH, HEIGHT = 600, 300

SQUARE_WIDTH, SQUARE_HEIGHT = 5, 5
WALL_WIDTH, WALL_HEIGHT = 20, 20

velocity = 0.15
kill_line_velocity = 0.09
results = Results()

pygame.init()

screen = pygame.display.set_mode((WIDTH, HEIGHT))

clock = pygame.time.Clock()


levels = [
    "BBBBBBBBBBBBBBBBBBBBBBBWWWWWWW",
    "B                      W     W",
    "B           WWWWWWWWWWWW  E  W",
    "B           W                W",
    "B           W      WWWWWWWWWWW",
    "B    WWWWWWWW      W         B",
    "B    W             W         B",
    "B    W            WW         B",
    "B    W            W          B",
    "WWWWWW        WWWWW          B",
    "W             W              B",
    "W             W              B",
    "W             W              B",
    "W             W              B",
    "WWWWWWWWWWWWWWWBBBBBBBBBBBBBBB"
]


left = 0
top = 0

walls = []
borders = []
exit = 0

for row in levels:
    for col in row:
        if col == 'W':
            wall = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
            walls.append(wall)
        if col == 'B':
            border = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
            borders.append(border)
        if col == 'E':
            exit = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
        left += WALL_WIDTH
    top += WALL_HEIGHT
    if left >= WIDTH:
        left = 0


font = pygame.font.Font('freesansbold.ttf', 12)
        
class GeneticAlgorithm:
    def __init__(self, populationSize, eliteSize, mutationRate, generations):
        #self.population = population
        self.populationSize = populationSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
        self.generations = generations
        self.currentPopulation = []
        self.bestFitnessMovimentations = 1
        self.bestDistance = 0

    def start(self):
        self.currentPopulation = self.initialPopulation(self.populationSize, 4, 2, 4)

        for i in range(0, self.generations):
            print('generation: ', i)
            results.incrementGenerations()
            self.currentPopulation = self.nextGeneration(self.currentPopulation, self.eliteSize, self.mutationRate)

        bestIndividualIndex = self.rankIndividuals(self.currentPopulation)[0][0]
        bestIndividual = self.currentPopulation[bestIndividualIndex]
        print(bestIndividual)
        
        
    def createPath(self, listDirections):
        return random.sample(listDirections, len(listDirections))


    def initialPopulation(self, populationSize, input_size, hidden_size, output_size):
        population = []
            
        for i in range(0, populationSize):
            individual = []
            individual.append(np.random.uniform(-1., 1., (input_size,hidden_size)))
            individual.append(np.random.uniform(-1., 1., (hidden_size,hidden_size)))
            individual.append(np.random.uniform(-1., 1., (hidden_size,output_size)))
            
            population.append(individual)
            
        return population


    def rankIndividuals(self, population):
        fitnessResults = {}
        
        fitnessResults = Fitness(population).getPathFitness()
        
        #for i in range(0, len(population)):
            #print('population', population[i])
            
            #print('fitness', fitnessResults)
        
        rank = sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
        
        #print('rank')
        #print(rank)
            
        return rank


    def selection(self, populationRanked, eliteSize):
        #print('populationRanked', populationRanked)

        selectionResults = []
        df = pd.DataFrame(np.array(populationRanked), columns=["Index", "Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100 * df.cum_sum / df.Fitness.sum()

        for i in range(0, eliteSize):
            selectionResults.append(populationRanked[i][0])
            
        top30Percent = int(len(populationRanked) * 0.3)
        
        firsts = populationRanked[:top30Percent]
        
        length = len(firsts)
        
        
        while len(selectionResults) < length:
            pick = 100 * random.random()
            
            for j in range(0, length):
                if pick <= df.iat[j, 3]:
                    selectionResults.append(populationRanked[j][0])
                    break
            
                    
        #print('selectionResults')
        #print(len(selectionResults))
        return selectionResults

    def matingPool(self, population, selectionResults):
        matingpool = []

        for i in range(0, len(selectionResults)):
            index = selectionResults[i]
            matingpool.append(population[index])
            
        #print('matingpool')
        #print(len(matingpool))
        #print(matingpool)
            
        return matingpool

    def breed(self, parent1, parent2):
        
        #print('parent1', parent1)
        #print('parent2', parent2)
        
        child = []
        childP1 = []
        childP2 = []
        
        
        # a reproducao vai ser diferente
        # obter cada array de pesos
        # mesclar as propriedades de cada array
        # adicionar ao filho cada array
        
        #weights = []
        
        for i in range(0, len(parent1)):
            
            
            
            sep = random.randint(1, len(parent1[i]) - 1)
            
            weights = []
            
            childP1 = parent1[i][:sep]
            childP2 = parent2[i][sep:]
                        
            #print('childP1', childP1)
            #print('childP2', childP2)
            
            for j in childP1:
                weights.append(j)
                
            for k in childP2:
                weights.append(k)
            
            child.append(weights)
            
            '''parentI = random.randint(1, 2)
            
            if parentI == 1:
                weights.append(parent1[i])
            if (parentI == 2):
                weights.append(parent2[i])
                
            child = weights'''
            
        #print('child', child)
        
            
        return child

    def breedPopulation(self, matingpool, eliteSize, currentGeneration):
        children = []
        
        #length = len(matingpool) - eliteSize
        length = len(currentGeneration) - eliteSize
        
        for i in range(0, eliteSize):
            children.append(matingpool[i])
            
        pool = random.sample(matingpool, len(matingpool))
        
        #sep = random.randint(1, len(matingpool))
        #pool = matingpool[:sep]
        
        #print('matingpool')
        #print(len(matingpool))
        
        #pool = random.sample(matingpool, length)
        
        #print('children pool')
        #print(pool)

        for i in range(0, length):
            index1 = random.randint(0, len(pool)-1)
            index2 = random.randint(0, len(pool)-1)
            parent1 = pool[index1]
            parent2 = pool[index2]
            child = self.breed(parent1, parent2)
            children.append(child)

        return children


    def mutate(self, individual, mutationRate):
        for i in np.arange(len(individual)):
            #print('mutate weight', i)
            for neuron in range(len(individual[i])):
                if (random.random() < mutationRate):
                    newWeight = []
                    for j in range(len(individual[i][neuron])):
                        newWeight.append(random.uniform(-1., 1))
                        #print('neuronio', individual[i][neuron])
                        
                    print('mutacao', newWeight)

                    individual[i][neuron] = newWeight

        return individual

    def mutatePopulation(self, population, mutationRate):
        mutatedPopulation = []

        for individual in range(0, len(population)):

            mutatedIndividual = self.mutate(population[individual], mutationRate)
            mutatedPopulation.append(mutatedIndividual)

        return mutatedPopulation

    def nextGeneration(self, currentGeneration, eliteSize, mutationRate):
        populationRanked = self.rankIndividuals(currentGeneration)
        #print('rank')
        #print(len(populationRanked))
        
        selectionResults = self.selection(populationRanked, eliteSize)
        
        matingpool = self.matingPool(currentGeneration, selectionResults)
        
        #print('matingpool')
        #print(len(matingpool))
        
        children = self.breedPopulation(matingpool, eliteSize, currentGeneration)
        
        #print('children')
        #print(len(children))
        
        return self.mutatePopulation(children, mutationRate) 


ga = GeneticAlgorithm(populationSize=100, eliteSize=2, mutationRate=0.05, generations=500)
ga.start()    
    
#while True:

    #events = pygame.event.get()
    
    #for event in events:
        #if event.type == pygame.KEYUP:
            #if event.key == pygame.K_SPACE:
          
                #ga = GeneticAlgorithm(populationSize=50, eliteSize=10, mutationRate=0.01, generations=100)
                #ga.start()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
generation:  0


/home/joeltonguerreiro/workspace/env_python3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: overflow encountered in exp


bestDistance 322.96749062405644
mutacao [-0.4475967332244082, -0.11934132444516887]
mutacao [0.7957551314558258, 0.9715930607555077]
mutacao [-0.043665707915867324, -0.47477020272252446]
mutacao [0.8510958708269094, -0.11794000740461308, -0.3426411148194173, 0.1425141409418509]
mutacao [0.2854013647340603, -0.2879345505231825]
mutacao [0.4549742077381118, 0.693899085355087]
mutacao [-0.8902250869460435, 0.33433393847950166]
mutacao [-0.999699839853275, 0.9742764207732277]
mutacao [-0.06512663871384339, -0.22277756388595926, 0.594333032593594, 0.02730969853312537]
mutacao [0.791177448229615, -0.6611968482028796, -0.3839302475329869, 0.22000842167669554]
mutacao [0.7632179879895673, 0.6189623472638823, -0.7377018868497385, 0.9843403959515182]
mutacao [-0.5622807159100547, -0.4677458452978971]
mutacao [-0.7607440986897622, 0.7078774817437887]
mutacao [0.21719210084891127, 0.9700080329106986]
mutacao [0.6923200098276545, 0.8849777837001149]
mutacao [0.22851903099652393, 0.3552283468375905]

bestDistance 322.18783341398847
mutacao [0.30149986892323977, -0.399237624409847]
mutacao [0.5820603249105221, -0.041206690462272766]
mutacao [-0.17362789946566415, 0.3733733086640234]
mutacao [-0.5389580461466721, 0.6662561235538988]
mutacao [0.7354569919581693, -0.6422864981736738]
mutacao [-0.3055143477012807, -0.8413941303767829]
mutacao [0.2774699181356446, 0.6319211154731552, -0.7258566519073752, 0.39601704629723233]
mutacao [-0.25937213124741754, -0.7098920659006478, -0.6267824106970643, -0.26862001041497074]
mutacao [-0.02480864967805907, 0.44013849246765724]
mutacao [-0.1768483552025859, 0.10529111360791732]
mutacao [-0.45209778106311993, -0.8578046608764145]
mutacao [-0.9359935860360593, -0.8425471270403642, 0.20264049233813242, 0.5383104509224343]
mutacao [0.6825438053844035, 0.984960145833587]
mutacao [-0.3270507660245252, 0.6371188042302667]
mutacao [-0.9654672427921469, -0.1790506180717013]
mutacao [-0.8006036771928859, 0.25231639596914723]
mutacao [0.3033753801010026, 0.

bestDistance 316.80909077865806
mutacao [-0.8063211446617977, 0.7458506635650404]
mutacao [-0.6119868581934158, -0.2600976455013928]
mutacao [0.3213400453140196, 0.8727810462574155, -0.7386236525511649, 0.5304462359126274]
mutacao [0.6709715637809099, -0.14752208086848473]
mutacao [0.887858095265075, 0.7371599581099924]
mutacao [0.3760896673827925, 0.5076711162352214]
mutacao [0.2939201101001414, -0.988726621884157]
mutacao [0.9219834210040327, -0.2168355534658002, 0.11575531989810384, 0.5487861441210491]
mutacao [-0.4717912082361382, 0.05849279050583789, 0.8402775263188329, 0.8981093391426525]
mutacao [-0.3211272631514006, 0.23248795518548326, -0.3582442214358996, 0.19072868846552682]
mutacao [0.9208724345204422, -0.9166200352285467]
mutacao [0.2694484587842356, -0.23515787918166642]
mutacao [0.1845116255619781, -0.4158151588343535]
mutacao [0.35613585661467084, 0.8636558206717839, -0.14552683511390896, -0.6968701418877723]
mutacao [0.7334569225377512, 0.2015281457550262]
mutacao [-0.

...

bestDistance 2.0
mutacao [-0.250908587421814, 0.44269385493199276]
mutacao [0.8261127146568117, 0.074946508335896]
mutacao [-0.6158510273288504, -0.09587098506462155]
mutacao [-0.5278224243402863, 0.6259635014428129, 0.39477422359252357, -0.19875515453398318]
mutacao [-0.6596960718120426, 0.0364267792518469]
mutacao [0.6056980720692722, -0.29002997001992004]
mutacao [-0.25002431778418854, 0.45269155129568395, 0.25453211058592284, 0.32605837228480405]
mutacao [-0.06350699549491634, 0.865251122413571]
mutacao [-0.9257148980248653, -0.03412102675491391]
mutacao [0.7472213991388177, -0.346716265743666, 0.9251872528436036, -0.9189503903567136]
mutacao [-0.8514055389285904, 0.47792874948388886]
mutacao [0.9292100285760085, 0.3823588062883474]
mutacao [-0.37742292672993605, -0.429201230020249, 0.23417017284588204, -0.28158882175297495]
mutacao [0.7018479631666816, 0.6935571034945605]
mutacao [0.6699531255068545, 0.09224116969700735]
mutacao [-0.22761398152193513, -0.2442973921824756]
mutacao 

bestDistance 2.0
mutacao [0.42348104310921575, -0.2637403157556273]
mutacao [-0.4595017840018627, 0.8664576750164561]
mutacao [-0.3986244195057751, 0.6315707543565738]
mutacao [0.23643658327872785, -0.6757701374259555]
mutacao [-0.2991307157964196, 0.07399886954673729]
mutacao [-0.04151246700191735, -0.8038972951604324]
mutacao [0.2816480753573358, 0.05110692470990075]
mutacao [-0.9699599272730273, -0.07452037093639707]
mutacao [-0.5108635729920235, -0.005215295585807134, -0.2940930154132937, 0.25406579056373735]
mutacao [-0.5691070471935582, 0.6426987502991173]
mutacao [0.7181438548068135, 0.15042081182688394]
mutacao [0.5762996442843504, -0.980204537357573]
mutacao [-0.1865904524867239, 0.4261638386504323]
mutacao [0.48745951810880084, -0.9267791707596684]
mutacao [-0.8704096679827944, 0.5834031852852799]
mutacao [-0.46499280367028084, 0.16045151576229566]
mutacao [-0.4179500580179405, 0.03542860939531023, 0.8313831000007816, -0.7176195558960567]
mutacao [0.7882889340254617, 0.148533

bestDistance 4.0
mutacao [-0.808473566052941, -0.9023807111463504]
mutacao [-0.14161074619589042, 0.41884223190713255]
mutacao [0.08949977869439518, -0.3781084933941705, -0.8176585090592723, 0.8543355235232717]
mutacao [-0.1698608543651725, 0.8551779398342911]
mutacao [-0.43319797334477705, -0.8246170684529863]
mutacao [0.7863361626080365, -0.356618113314745, -0.1587385715943037, -0.9640496370231038]
mutacao [-0.44455600172725274, 0.6849182448833535, -0.21336993861508535, 0.20105874029636994]
mutacao [0.44747088574708815, 0.9545829513974757]
mutacao [-0.07359921071965991, 0.5367899963003635]
mutacao [0.39835482270142264, -0.3953254786279208, -0.828614651710996, -0.11244757759184965]
mutacao [0.17001211901530766, -0.3032495866357494, -0.738178188820173, -0.6100465163414266]
mutacao [0.690102786898279, 0.5498496229667149]
mutacao [-0.054354553785860515, -0.586422679950096]
mutacao [-0.9264427540419535, -0.05808206378616254]
mutacao [-0.7765820323899775, 0.6517087228893712, 0.297015523445

bestDistance 6.0
mutacao [-0.11254853345861782, -0.4936211292660224]
mutacao [-0.922122126172245, 0.8181680709460442]
mutacao [-0.7992399928915723, -0.11303258097002788]
mutacao [-0.05391353673295973, -0.7338301612791609]
mutacao [-0.11396144763320781, -0.5777643658700067]
mutacao [-0.1565639763587363, 0.5833534872945627, 0.17982618033977227, -0.3735604441011333]
mutacao [-0.5514769820861205, 0.4947819398669451, 0.1940236416912775, -0.5046619633080422]
mutacao [-0.19767020795662216, 0.5937151381670454]
mutacao [0.9503779358014479, -0.7213918319386148]
mutacao [-0.6454318241329899, -0.3122061431254761, -0.9573500965464359, 0.7595422367736584]
mutacao [-0.5011837776445849, 0.7249580354747451]
mutacao [0.22660746505014506, -0.49942745541642286]
mutacao [-0.7932603056394791, 0.9083106848439957]
mutacao [0.96444523235259, -0.837286314980052]
mutacao [0.6301134524605672, 0.09605992922453832]
mutacao [-0.7199474533752457, 0.6799164258495549, 0.18166621050881626, 0.10433719277831988]
mutacao [

bestDistance 1.0
mutacao [0.021195712634235697, 0.7635825143416077, 0.27314294194576805, -0.5928344567245691]
mutacao [0.6699864740820434, 0.09918945279011138]
mutacao [-0.13817450852019086, -0.3556759561814766]
mutacao [0.021682040304470407, 0.7161701944665202]
mutacao [-0.6085157093500679, -0.43427076259450903, -0.7638936718252127, -0.3310678229652615]
mutacao [0.7378811404102756, -0.5962280447169213]
mutacao [-0.9482921673196765, 0.4516838657696314, 0.6108022323150122, 0.648683989434164]
mutacao [0.9912055844253549, -0.26328756036245604]
mutacao [0.6600928217366586, 0.7217856999247121]
mutacao [0.3618993747221455, 0.9491563061883266]
mutacao [-0.7523720910511653, 0.08204343809991221]
mutacao [0.6897503509921457, 0.05427905123123211]
mutacao [0.052174444995602975, 0.9259680422794185, 0.8616085843735832, -0.24197205542053046]
mutacao [0.6538930858139609, -0.012145749490352964]
mutacao [0.691982947044371, -0.969994494933164]
mutacao [0.6460277689252447, -0.8667819565985286]
mutacao [-0